<a href="https://colab.research.google.com/github/Tunrham/UTS_ML2019_ID11403868/blob/master/A2_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
################################################################################
########################## Student Number: 11403868 ############################
########################### Subject Number: 31005  #############################
################################################################################
##################### Below is Model Code used creating a ######################
################### LSTM Predicting Tennis Rally Sequences #####################
################################################################################

In [0]:
################################################################################
############################## Assignment Links ################################
################################################################################
### Code: https://github.com/Tunrham/UTS_ML2019_ID11403868/blob/master/A2_Model_Code.ipynb
### Youtube Link: 

# Practical Machine Learning Project: Using LSTM for Pattern Recognition in Tennis


## Introduction

### Introduction
 


The role of analytics is historically documented in sports, from Bill James' Sabremetrics for Baseball in the 1980's, to analytics' role today in multi-billion dollar sports leagues including the National Basketball Association and the English Premier League. Tennis however, has always lagged behind - with data rarely publically available, and any data solutions occuring quietly, behind closed doors. 

In this Project, I am attempting to implement a Recurrent Neural Network (RCC) architecture of Long Short-Term Memory (LSTM) to identify rally patterns in professional Tennis matches - in an effort to implement Machine Learning solutions to Tennis players at a high level. 

### Algorithm Implementation



RCCs are Artificial Neural Networks designed for pattern recognition. They are adept identifying sequential data, using Backpropagation Through Time. This takes the current input value, and previously seen input values to give an output value.

LSTM based Neural Networks were posed as a solution to a common RCC issue - the Vanishing Gradient Problem. This was a Backpropagation problem, occurring when the Neural Network's loss function approached zero as more input layers were added - making model training difficult. LSTM's combat the Vanishing Gradient Problem by storing information outside the current Reccurent Network - aiding current sequence input analysis through remembering (or forgetting) similar imput patterns from previous network training iterations. 

For this project, the input value is going to be the first shot in a Tennis Rally (A serve). The output, will be the resultant five shots (based on the input). This number was chosen as it has been identified that a Server's advantage is lost around the 4th to 8th shot in a rally. Rally data will be formatted consistent with the Match Charting Project metadata standards.  

## Exploration

### Dataset



The dataset contains point by point information from 169 professional tour level Tennis matches from 2001-2018, sourced from the Tennis Match Charting Project Github page. The dataset contains 57 different attributes and around 26,000 points, including: Player Names, Current Serving Player, Current Score, Rally Length, Point Winner, and, Rally Data.

#### Rally Data Explained

Rally information is the most important attribute from the dataset, accounting for the LSTM Input and Output. An example rally is below;  

#### Rally Example

4+f18v3b3w@

#### Rally Example Explanation

The above code represents a four-shot rally:
- **4+**
  - PlayerA hits a Serve out Wide on the Duece Court and runs towards net
- **f18**
  - PlayerB hits a moderately deep Forehand Return towards PlayerA's Forehand side (known as 'cross-court' in Tennis)
- **v3**
  - PlayerA hits a Forehand Volley from the net into PlayerB's Backhand court
- **b3w@**
  - PlayerB hits a Backhand towards PlayerA's backhand, missing the court wide and out
  
  
Each dataset row contains similar sequences of metadata representing rallies. 

### Dataset Preparation



The preparation of the dataset for RCC suitability can be broken into a Preprocessing phase and a Data Manipulation phase

#### Preprocessing Phase
This phase involved identifying relevant dataset input for LSTM modelling. Many steps here were formulated for solving Tennis related questions.

##### Eliminating Left-Handed Players from the Dataset
Left handed-players were removed from the dataset for two reasons - the first, to standardize the represented player type. The second, for simplicity of data representation - due to Left-Handed players requiring different coded rally notation. Keeping these players would have led to sample size errors as well as confusing LSTM output notation.   

##### Eliminating Rallies where the First Serve Wasn't Legal
If a player misses their First Serve of a point, they get a second attempt. All points where the player missed their First Serve were removed from the dataset. This was in an effort to focus on rally patterns resulting from a First Serve only.

##### Eliminating Rallies Under Two Shots in Length
Rallies under two shots in length are usually ended by a good Serve, or an unnaturally good Return of Serve. These points are neither interesting for Tennis Knowledge, or, useful for LSTM Pattern Recognition - therefore, these points were removed.

##### Splitting the Dataset into Return Focused Data by Court Position
This was the largest piece of preprocessing, involving splitting the dataset into two sub-segments. This allowed for a more tailored LSTM approach, focusing entirely on the data from the Returning Player's point of view.



The major segment only used points won by the Returner, sub-segmented by points split by either of the starting court positions (Duece Side, or Advantage Side). Therefore, the final datasets were:
- The Returner Won the Point, Returning from the Duece Side (2017 rows)
- The Returner Won the Point, Returning from the Advantage Side (2685 rows)

 
This step was important, due to the notation used for rally sequences. For example, the code "6" refers to hitting a serve through the centre of the court (known as the "Tee"). If a Server hits a Tee serve on the Duece Court, a right-handed Returner can only hit a Backhand (coded as "b"). However, if the Server hits a Tee serve (still coded "6") on the Advantage Court, the Returner can only hit a Forehand (coded as "f"). Therefore, segmenting the data by side prevented nonsensical LSTM output derived from the inability to differentiate which side of the court the point started.  


#### Data Manipulation Phase


The Data Manipulation Phase involved formatting the Rally data for Tokenization suitability, and usability in LSTM modelling. The previous rally data formatting was one large string, however, it needed to be sequential (structured similarly to a sentence). An Ad Hoc function was created to space rally data accordingly. After this structuring, a line break ("\n") was added to each row for future formatting purposes. Using this function, the Rally Example from above became;

4+ f18 v3 b3w@\n




## Methodology

### Padded Rally Sequences
To analyse rally patterns for Returning players, a Line-by-Line LSTM model was used. This model splits each rally sequentially shot-by-shot, observing how previous shots have a cause and effect relationship with future ones. This method also eliminates ambiguity where one shot can have multiple different output values.

This format was used for Returners only, attempting to find a "pseudo" counter strategy (output as a rally sequence) to common Serve directions (input).


### Padded Rally Sequences Code
Below is the model code used. The input data is not included below, and can be found with the full code (provided above).

The LSTM Model is broken into the following sections:


#### Creating a Function to Generate Output Sequence
The first step involves wrapping our LSTM model into a function for ease of use.

In [0]:
### Create a Function For Generating a Sequence Given our Model
### and Input Stimulus 
def generate_seq(model, tokenizer, max_length, first_serve, n_shots):
	in_text = first_serve
	# Below Generates a fixed number of words for Sequencing
	for _ in range(n_shots):
		# "Encoded" creates sequences of words from the base stimulus
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# Below pads the created sequences so that they are equi-length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# This is a Keras module for predicting the likelihood of the next word
    # given a group of words
		yhat = model.predict_classes(encoded, verbose=0)
		# This maps the output of a word from an integer value to the actual word.
    # In our example this will give us a Rally Token
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# This appends the predicted output to the input values
    # (based on predictions from the model)
		in_text += ' ' + out_word
	return in_text


#### Sourcing and Tokenizing the Data
Next, the data is Tokenized. This involves converting a string to a mapped integer. These integers can then be binarised so that the RCC can use them in the hidden layers. 

In [0]:
# Source and Tokenize Data
# Here we are using our Returner Data on the Duece Side
data = ds_r_w # Again, important to note that this dataset is not provided here
# The Keras Tokenizer Module will tokenize our data to mapped integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#### Determining Vocabulary Size
Next, we identify the vocabulary size and the number of unique words (or in this case, Rally Shots). This is an important step in the cleaning process for LSTM models. If running LSTM on a corpus, this step would involve Lower Casing all words for normalization. This is important as a more succinct allows for faster model training. In our use case however, the data is already Lower Cased, and the datasets are reasonably small.

In [0]:
# The next step is to determine the Vocabulary Size (number of unique words)

vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

#### Padding Sequences
Sequence Padding involves making sure each sequence is equal length. This is a feature that is necessary whilst using the Keras Library for RCC's. In the model, the sequences have been Padded to match the Maximum Length Sequence found in the dataset. 

##### Sequence Padding Example
Sequence Padding is used to make all Token Sequences an equal length. So, if the model has three Tokenized Sequences, represented as:

[15, 8, 9, 1, 2]

[15, 7, 9]

[16, 8, 1, 23, 29, 5, 8]

Post Padding, the Sequences would become:

[0, 0, 15, 8, 9, 1, 2]

[0, 0, 0, 0, 15, 7, 9]

[16, 8, 1, 23, 29, 5, 8]

In [0]:
# Create and Pad Sequences
# Below will create sequences of equal length for modelling
sequences = list()
for line in data.split('\n'):# This is why we used "\n" earlier, for splitting
	encoded = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(encoded)):
		sequence = encoded[:i+1]
		sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

# We then need to pad our sequences so that they are all the same length
# This is a Keras Only Requirement, padding to our maximum length sequence. 
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

#### Splitting Sequences into X and Y values

Next, we need to split our generated Sequences into Input and Output elements (or X and Y values). 

In [0]:
# This splits our sequences into X and Y values
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)

#### Creating the LSTM Model
For the LSTM Neural Network, the Keras Sequential model is used. A Sequential Model is defined as a Linear Stack of layers. The Softmax Activation function was chosen over the Sigmoid Activation function due to the Multiclass nature of the Tennis Dataset.

In [0]:
# Here we Create our model. We are using a Sequential LSTM (using Softmax)
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

#### Fitting the LSTM Model on Training Data
The final section fits the Model using the Training Data. I used 625 Epochs (or total iterations through the entirety of the Training Data) however this may be unreasonable given Dataset sample sizes. 

In [0]:
# Next we must fit our model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the Network given Training Data (625 Epochs takes a really long time!)
model.fit(X, y, epochs=625, verbose=2)

##### Loss Function, Optimizer, and Metrics used
A Categorical Crossentropy Loss function was chosen over a Binary Crossentropy Loss Function due to the Multiclass nature of the Dataset. The Adam Optimization method (derived from "Adaptive Movement Estimation") was chosen instead of a more classical Stochastic Gradient Descent Optimization method, as Adam has been shown to outperform similar methods over comparable dataset iterations. Finally, using "Accuracy" for metrics will print the mean accuracy rate per Token Sequence prediction each iteration of the learning process. 

#### Generating Output from the Model:
To generate output, simply run the initially created function, inputting a rally shot code. For example, given the above code, and the following code:


In [0]:
# Output from model
print(generate_seq(model, tokenizer, max_length-1, '6', 5))
print(generate_seq(model, tokenizer, max_length-1, '5', 5))
print(generate_seq(model, tokenizer, max_length-1, '4', 5))

Output:




6 b28 f1 f1 f1 f1




5 b28 f1 f1 f3 s3




4 f29 b2 f1 f1 f1






The above output was for points based on the Duece Side. If we ran the same code for the Advantage side, we would get:


6 f28 f3 b3 b2 f3

5 f28 f3 s2 f2d s3

4 b28 f1 f2 f3 s3

## Evaluation

### Output from Model
After running the model on both the Duece Side and Advantage Side Datasets, output for common rally sequences for Six and Ten shot rallies given an input value can be seen below for each side:




#### Output for Common Shots in Rallies on the Duece Side 

**With Regular Serves (6 Shot Sequences)**

6 b28 f3 b3 b3 s3

5 b28 f 3 b3 i1

4 f28 b2 b2 f2 f2

**Rally Sequences (6 Shot Sequences)**

b3 b3 b3 b2 f3 b3

b2 f 1 f1 v3 m3

b1 f2 f2 b2 f3 b3

**With Regular Serves (10 Shot Sequences)**

6 b28 f3 b3 b3 s3 b3 b2 f2 f2

5 b28 f 3 b3 i1 f 3 m2 o3

4 f28 b2 b2 f2 f2 f2 b2 f1w f1

**Rally Sequences (10 Shot Sequences)**

b3 b3 b3 b2 f3 b3 b2 b2 b3 b2

b2 f 1 f1 v3 m3 p1 v3 s3w v1

b1 f2 f2 b2 f3 b3 b2 f3 b3 b2

#### Output for Common Shots in Rallies on the Advantage Side

**With Regular Serves (6 Shot Sequences)**

6 f28 f3 b3 b2 f1

5 f28 f3 b3 b2d f3

4 b38 s3 f 1 f1


**Rally Sequences (6 Shot Sequences)**

b3 b3 b3 b3 b3 b3

b2 f3 b2u3 zn b3 z2

b1 f1 f3 b3 b3 f3


**With Regular Serves (10 Shot Sequences)**

6 f28 f3 b3 b2 f1 f1 f1 f2 s3

5 f28 f3 b3 b2d f3 s2n f 3 m

4 b38 s3 f 1 f1 v3 b3 z2 l2


**Rally Sequences (10 Shot Sequences)**

b3 b3 b3 b3 b3 b3 b3 b3 b3 b3

b2 f3 b2u3 zn b3 z2 f1 v3w b 1d

b1 f1 f3 b3 b3 f3 b3 b3 b3 b1

### Evaluation of Output



The output generated from the LSTM Model is fairly representative and realistic, echoing common Rally Sequences - for example, a lot of output rallies involved multiple cross-court shots, a regular pattern of play at high-level Tennis. 

Some model observations for Tennis players are:
- _The Returner should aim for a deeper return of Serve_ - the output of the model always preferred deeper serve returns (regardless of Serve direction) over shorter ones. This suggests that if a Returner can hit a deeper Return of Serve, they are more likely to win the point.
- _The Importance of Cross-Court Rally Sequences, Especially towards the Backhand side of the Court_ - the output of the model heavily featured shots played towards the Backhand corner of the court. This should be a major focus area for Tennis players. 

The model output also has some areas where it is somewhat nonsensical:
- _Occasionally output will be split between shot and direction_ - for example "f" followed by "1" instead of "f1"
- _Once rallies start going past a certain length they lose sensibility_ - this includes missed shots, or continuous looping rallies
- _The model is unable to identify Net points_ - The model is unable to decipher how a player approaching the net forces the next shot that player hits to also be at the net. I believe this is a sample error, due to net play being uncommon at high-level play.

### Evaluation of Model and Comparative Study



The earlier Preprocessing steps allowed the model to differentiate Returner shot options based on previous shot and side of court, certainly validating the preprocessing methodology used. 


One negative of the model was the number of training epochs used - being too time consuming relative to marginal gain per epoch. The Accuracy Rate and Loss Rate by number of Epochs (for the Duece Side dataset) are shown below:

![Accuracy by Training Epochs](https://drive.google.com/uc?id=1MY9LhhFYiWs_JyqdWdToWzyMGxyqIW54)


![Loss by Training Epochs](https://drive.google.com/uc?id=1n3wP_65uV-l9U--94g55IStjAm3fEQSS)

## Conclusion


### Conclusion
This Model doesn't give infallible Tennis Rally Sequence output for beating strong Serving players. The model is however, a good representation of the algorithmic usage of Long-Short Term Memory Recurrent Neural Networks, and would be better suited for tasks including Text Recognition, and (given enough data) Pattern Recognition in Price Based Markets.


### Possible Improvements

If I were to pursue the idea of using LSTM for Tennis Modelling further, a larger dataset would be necessary - due to the large number of unique Rally Sequences occuring in Tennis, and, also to the difficulty of modelling a sport containing an abundance of relevant input values. I would also try and normalize the rally length distribution more, allowing for a lower padding sequence length in model training.

Given this, I recommend for Tennis Analysis, a different, more exploratory and visualisation based approach undertaken when attempting to use Analytics and Machine learning in this space. 

  

## Ethical


I believe that if we are to analyse Machine Learning Algorithms such as LSTM, they need to be observed under an umbrella of the "Common-Good Approach" of ethical decision making. This approach refers to any action undertaken being done with intent for the betterment of society as a whole.

Pattern Recognition algorithms are already used efficiently in the medical space - featuring in areas indlucing: disease diagnosis, cancer detection, and even in hospital bed occupation forecasting. Methodologies such as these are far better at pattern recognition across sample spaces than humans are. There is a caveat however, that some human interaction is necessary when using machine learning to preserve human existence. The validation of the methodology and results of Machine Learning needs to be managed and monitored so that it is not misused, or misunderstood. Misconstruing information, or misusing input methodology leading to bad output data are problems that can have real human consequences. Whilst working symbiotically with Machine Learning, we need to ensure that we truly understand the methodology of using algorithmic solutions, and how the output can be properly used and managed if we wish to ethically pursue this area. 

A general rule of thumb, is that a computer shouldn't be able to make a decision that is inexplicable as a human - there needs to be a point of contact between the two.  


## Video Pitch